In [1]:
# Cross referencing Direct and Anon Groups between neutral and inflam groups

In [2]:
# Import common libraries
import pandas as pd
from itertools import compress
from collections import defaultdict
from copy import deepcopy
from pprint import pprint
import os

# Import MNE processing
from mne.viz import plot_compare_evokeds
from mne import Epochs, events_from_annotations, set_log_level

# Import MNE-NIRS processing
import mne
from mne_nirs.channels import get_long_channels
from mne_nirs.channels import picks_pair_to_idx
from mne_nirs.datasets import fnirs_motor_group
from mne.preprocessing.nirs import beer_lambert_law, optical_density,\
    temporal_derivative_distribution_repair, scalp_coupling_index
from mne_nirs.signal_enhancement import enhance_negative_correlation

# Import MNE-BIDS processing
from mne_bids import BIDSPath, read_raw_bids

# Import StatsModels
import statsmodels.formula.api as smf

# Import Plotting Library
import matplotlib.pyplot as plt
import seaborn as sns

# Set general parameters
set_log_level("WARNING")  # Don't show info, as it is repetitive for many subjects

ignore = [".DS_Store", "sub-03"]

# Based on Guides Published Here
# https://mne.tools/mne-nirs/stable/auto_examples/general/plot_16_waveform_group.html

In [3]:
def individual_analysis(bids_path):

    # Read data with annotations in BIDS format
    # raw_intensity = read_raw_bids(bids_path=bids_path, verbose=False)
    raw_intensity = mne.io.read_raw_snirf(bids_path, verbose=True, preload=False)
    raw_intensity = get_long_channels(raw_intensity, min_dist=0.01)
    
    channel_types = raw_intensity.copy()
    print(channel_types)
    
    raw_intensity.annotations.rename({'4': 'Control', '2': 'Neutral', '3': 'Inflammatory', '1':'Practice'})

    # Convert signal to optical density and determine bad channels
    raw_od = optical_density(raw_intensity)
    sci = scalp_coupling_index(raw_od, h_freq=1.35, h_trans_bandwidth=0.1)
    raw_od.info["bads"] = list(compress(raw_od.ch_names, sci < 0.5))
    # raw_od.interpolate_bads()

    # Downsample and apply signal cleaning techniques
    raw_od.resample(0.8)
    raw_od = temporal_derivative_distribution_repair(raw_od)

    # Convert to haemoglobin and filter
    raw_haemo = beer_lambert_law(raw_od, ppf=0.1)
    raw_haemo = raw_haemo.filter(0.02, 0.3,
                                 h_trans_bandwidth=0.1, l_trans_bandwidth=0.01,
                                 verbose=False)

    # Apply further data cleaning techniques and extract epochs
    raw_haemo = enhance_negative_correlation(raw_haemo)
    # Extract events but ignore those with
    # the word Ends (i.e. drop ExperimentEnds events)
    events, event_dict = events_from_annotations(raw_haemo, verbose=False)

    epochs = Epochs(raw_haemo, events, event_id=event_dict, tmin=-5, tmax=30,
                    reject=dict(hbo=200e-6), reject_by_annotation=True,
                    proj=True, baseline=(None, 0), detrend=0,
                    preload=True, verbose=False)

    return raw_haemo, epochs

In [4]:
# Isolate Evoked Anon Data

In [5]:
from ramda import *
all_evoked_anon = defaultdict(list)

anon_subjects = os.listdir("../BIDS_Anon/")

for sub in anon_subjects:
    if sub not in ignore:
        # Create path to file based on experiment info
        anon_f_path = f'../BIDS_Anon/{sub}/nirs/{sub}_task-AnonCom_nirs.snirf'

        # Analyse data and return both ROI and channel results
        raw_haemo, epochs = individual_analysis(anon_f_path)
        
        # Save individual-evoked participant data along with others in all_evokeds
        for cidx, condition in enumerate(epochs.event_id):
            all_evoked_anon[condition].append(epochs[condition].average())

Loading /Users/nolanbrady/Desktop/LabResearch/IndependentStudy/DataAnalysis/FADataAnalysisScripts/../BIDS_Anon/sub-06/nirs/sub-06_task-AnonCom_nirs.snirf
<RawSNIRF | sub-06_task-AnonCom_nirs.snirf, 36 x 18358 (1804.5 s), ~5.2 MB, data loaded>
Loading /Users/nolanbrady/Desktop/LabResearch/IndependentStudy/DataAnalysis/FADataAnalysisScripts/../BIDS_Anon/sub-07/nirs/sub-07_task-AnonCom_nirs.snirf
<RawSNIRF | sub-07_task-AnonCom_nirs.snirf, 36 x 17825 (1752.1 s), ~5.1 MB, data loaded>
Loading /Users/nolanbrady/Desktop/LabResearch/IndependentStudy/DataAnalysis/FADataAnalysisScripts/../BIDS_Anon/sub-05/nirs/sub-05_task-AnonCom_nirs.snirf
<RawSNIRF | sub-05_task-AnonCom_nirs.snirf, 36 x 17062 (1677.1 s), ~4.8 MB, data loaded>


In [6]:
# Isolate Evoked Direct Data

In [7]:
all_evoked_direct = defaultdict(list)

direct_subjects = os.listdir("../BIDS_Direct/")

for sub in direct_subjects:
    if sub not in ignore:
        # Create path to file based on experiment info
        direct_f_path = f'../BIDS_Direct/{sub}/nirs/{sub}_task-AnonCom_nirs.snirf'

        # Analyse data and return both ROI and channel results
        raw_haemo, epochs = individual_analysis(direct_f_path)
        
        # Save individual-evoked participant data along with others in all_evokeds
        for cidx, condition in enumerate(epochs.event_id):
            all_evoked_direct[condition].append(epochs[condition].average())

Loading /Users/nolanbrady/Desktop/LabResearch/IndependentStudy/DataAnalysis/FADataAnalysisScripts/../BIDS_Direct/sub-01/nirs/sub-01_task-AnonCom_nirs.snirf
<RawSNIRF | sub-01_task-AnonCom_nirs.snirf, 36 x 17098 (1680.6 s), ~4.9 MB, data loaded>
Loading /Users/nolanbrady/Desktop/LabResearch/IndependentStudy/DataAnalysis/FADataAnalysisScripts/../BIDS_Direct/sub-02/nirs/sub-02_task-AnonCom_nirs.snirf
<RawSNIRF | sub-02_task-AnonCom_nirs.snirf, 36 x 14804 (1455.1 s), ~4.2 MB, data loaded>
Loading /Users/nolanbrady/Desktop/LabResearch/IndependentStudy/DataAnalysis/FADataAnalysisScripts/../BIDS_Direct/sub-04/nirs/sub-04_task-AnonCom_nirs.snirf
<RawSNIRF | sub-04_task-AnonCom_nirs.snirf, 36 x 16843 (1655.6 s), ~4.8 MB, data loaded>


In [8]:
# Create an Anon Dataframe

In [9]:
anon_df = pd.DataFrame(columns=['ID', 'Chroma', 'Condition', 'Value'])

for idx, evoked in enumerate(all_evoked_anon):
    subj_id = 0
    for subj_data in all_evoked_anon[evoked]:
        subj_id += 1
        for chroma in ["hbo", "hbr"]:
            data = deepcopy(subj_data)
            value = data.crop(tmin=5.0, tmax=7.0).data.mean() * 1.0e6

            # Append metadata and extracted feature to dataframe
            this_df = pd.DataFrame(
                {'ID': subj_id, 'Chroma': chroma,
                 'Condition': evoked, 'Value': value}, index=[0])
            
            anon_df = pd.concat([anon_df, this_df], ignore_index=True)
            
anon_df.reset_index(inplace=True, drop=True)
anon_df['Value'] = pd.to_numeric(anon_df['Value'])  # some Pandas have this as object
anon_df['Group'] = "Anon"

In [10]:
direct_df = pd.DataFrame(columns=['ID', 'Chroma', 'Condition', 'Value'])

for idx, evoked in enumerate(all_evoked_direct):
    subj_id = 0
    for subj_data in all_evoked_direct[evoked]:
        subj_id += 1
        for chroma in ["hbo", "hbr"]:
            data = deepcopy(subj_data)
            value = data.crop(tmin=5.0, tmax=7.0).data.mean() * 1.0e6

            # Append metadata and extracted feature to dataframe
            this_df = pd.DataFrame(
                {'ID': subj_id, 'Chroma': chroma,
                 'Condition': evoked, 'Value': value}, index=[0])
            
            direct_df = pd.concat([direct_df, this_df], ignore_index=True)
            
direct_df.reset_index(inplace=True, drop=True)
direct_df['Value'] = pd.to_numeric(direct_df['Value'])  # some Pandas have this as object
direct_df['Group'] = "Direct"

In [11]:
# Comparing Control between groups

anon_input_data = anon_df.query("Condition in ['Control']").query("Chroma in ['hbo']")
direct_input_data = direct_df.query("Condition in ['Control']").query("Chroma in ['hbo']")

input_data = pd.concat([anon_input_data, direct_input_data])

model = smf.mixedlm("Value ~ Group", input_data, groups=input_data["ID"]).fit()
model.summary()

  ID Chroma Condition     Value   Group
0  1    hbo   Control -0.408635    Anon
2  2    hbo   Control -0.412552    Anon
4  3    hbo   Control -0.224928    Anon
0  1    hbo   Control -0.082096  Direct
2  2    hbo   Control -0.619755  Direct
4  3    hbo   Control -0.620592  Direct


/Users/nolanbrady/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/nolanbrady/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/Users/nolanbrady/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
          Mixed Linear Model Regression Results
==========================================================
Model:               MixedLM  Dependent Variable:  Value  
No. Observations:    6        Method:              REML   
No. Groups:          3        Scale:               0.0535 
Min. group size:     2        Log-Likelihood:      -0.9445
Max. group size:     2        Converged:           Yes    
Mean group size:     2.0                                  
----------------------------------------------------------
                Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------
Intercept       -0.349    0.134 -2.594 0.009 -0.612 -0.085
Group[T.Direct] -0.092    0.189 -0.488 0.626 -0.462  0.278
Group Var        0.001    0.316                           
==========================================================

"""

In [12]:
# Comparing Neutral between groups

anon_input_data = anon_df.query("Condition in ['Neutral']").query("Chroma in ['hbo']")
direct_input_data = direct_df.query("Condition in ['Neutral']").query("Chroma in ['hbo']")

input_data = pd.concat([anon_input_data, direct_input_data])

model = smf.mixedlm("Value ~ Group", input_data, groups=input_data["ID"]).fit()
model.summary()

/Users/nolanbrady/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/nolanbrady/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/Users/nolanbrady/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/Users/nolanbrady/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


<class 'statsmodels.iolib.summary2.Summary'>
"""
          Mixed Linear Model Regression Results
==========================================================
Model:               MixedLM  Dependent Variable:  Value  
No. Observations:    6        Method:              REML   
No. Groups:          3        Scale:               0.4294 
Min. group size:     2        Log-Likelihood:      -5.0838
Max. group size:     2        Converged:           Yes    
Mean group size:     2.0                                  
----------------------------------------------------------
                Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------
Intercept       -1.018    0.378 -2.689 0.007 -1.759 -0.276
Group[T.Direct]  0.492    0.535  0.919 0.358 -0.557  1.540
Group Var        0.000                                    
==========================================================

"""

In [13]:
# Comparing Neutral between groups

anon_input_data = anon_df.query("Condition in ['Inflammatory']").query("Chroma in ['hbo']")
direct_input_data = direct_df.query("Condition in ['Inflammatory']").query("Chroma in ['hbo']")

input_data = pd.concat([anon_input_data, direct_input_data])

model = smf.mixedlm("Value ~ Group", input_data, groups=input_data["ID"]).fit()
model.summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
          Mixed Linear Model Regression Results
==========================================================
Model:               MixedLM  Dependent Variable:  Value  
No. Observations:    6        Method:              REML   
No. Groups:          3        Scale:               0.8594 
Min. group size:     2        Log-Likelihood:      -6.8454
Max. group size:     2        Converged:           Yes    
Mean group size:     2.0                                  
----------------------------------------------------------
                Coef.  Std.Err.   z    P>|z| [0.025 0.975]
----------------------------------------------------------
Intercept       -0.401    0.593 -0.677 0.498 -1.563  0.761
Group[T.Direct]  0.648    0.757  0.856 0.392 -0.836  2.132
Group Var        0.195    0.953                           
==========================================================

"""